In [126]:
import requests
from bs4 import BeautifulSoup
from lxml import etree
import re

# url 


In [137]:
class Ebook:
    def __init__(self):
        self.start_url = "https://facert.gitbooks.io/python-data-structure-cn/"
    
    def download(self, url, bs = None, lxml = None, readyToHtml = None ):
        if bs == None and lxml == None:
            return Exception('发生错误，你必须选择一个解析器')
        if bs:
            bs = BeautifulSoup(requests.get(url).content.decode('utf8'))
            self.bs = bs
            return bs
        if lxml and readyToHtml is None:
            html = etree.HTML(requests.get(url).content.decode('utf8'))
            self.html = html
            return html
        if  readyToHtml:
            bs_html = BeautifulSoup(requests.get(url).content.decode('utf8'))
            content =  str(bs_html.find("section", 'normal markdown-section'))
            filter_href = re.sub(r'href="(?!http)', 'href="'+url, content)
            filter_src = re.sub(r'src="(?!http)', 'src="'+url, filter_href)
            return filter_src
    
    def find_all_a(self):
        if self.html is not None:
            html = self.html
            all_a_text = [i.strip() for i in html.xpath("//li[@class='chapter ']/a/text()")]
            all_a = html.xpath("//li[@class='chapter ']/a/@href")
            return all_a_text, all_a
        if self.bs is not None:
            bs = self.bs
            bs_li = bs.find_all("li", "chapter")
            all_a_text = []
            for li in bs_li:
                 all_a_text.append(''.join(li.find("a").string))
            return all_a_text
        
    def to_html(self, a_text, a_href):
        for text, href in zip(a_text, a_href):
            with open('html/'+text+'.html', 'w', encoding='utf8') as f:
                content = self.download(lxml = True, readyToHtml = True, url = self.start_url+href)
                f.write(content)
                 
        

In [138]:
e = Ebook()


In [139]:
e.download(e.start_url, lxml=True)

<Element html at 0x128e22b4348>

In [141]:
a_text, a_href = e.find_all_a()

In [142]:
e.to_html(a_text, a_href)

F:\miniconda\envs\tmp_example\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file F:\miniconda\envs\tmp_example\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


AttributeError: 'Ebook' object has no attribute 'html'

In [1]:
import asyncio, aiohttp
from asyncio import Queue
from bs4 import BeautifulSoup
from lxml import etree
import requests
import aiofiles
import re

In [2]:
class aioEbook:
    def __init__(self, loop):
        self.loop = loop
        self.queue = Queue()
        self.start_url = "https://facert.gitbooks.io/python-data-structure-cn/"
    
    async def find_all_a(self):
        async with aiohttp.ClientSession() as session:
            async with session.get(self.start_url) as resp:
                content = await resp.text()
                html = etree.HTML(content)
                all_a_text = [i.strip() for i in html.xpath("//li[@class='chapter ']/a/text()")]
                all_a = html.xpath("//li[@class='chapter ']/a/@href")
                for a_text, a_href in zip(all_a_text, all_a):
                    self.queue.put_nowait((a_text, a_href))
        
        tasks = [asyncio.Task(self.download_html(self.queue, self.loop)) for _ in range(30)]
        #await self.queue.join()
        await self.queue.join()
        for t in tasks:
            t.cancel()
            
    async def download_html(self, queue, loop):
        while self.queue:
            text, url = await self.queue.get()
            print(text, url)
            
            async with aiohttp.ClientSession(loop = loop) as session:
                async with session.get(self.start_url+url) as resp:
                    resp_text = await resp.text(encoding = 'utf8')
                    bs_html = BeautifulSoup(resp_text, 'lxml')
                    print('bs_html: ', bs_html)
                    content =  str(bs_html.find("section", 'normal markdown-section'))
                    filter_href = re.sub(r'href="(?!http)', 'href="'+url, content)
                    filter_src = re.sub(r'src="(?!http)', 'src="'+url, filter_href)
                    print('filter_src: ', filter_src)
            async with open('html2/'+text+'.html', 'w', encoding='utf8') as f:
                await f.write(filter_src)
            print('qsize(): ', self.queue.qsize())
            #queue.task_done()

In [ ]:
loop = asyncio.get_event_loop()
e = aioEbook(loop)
loop.run_until_complete(e.find_all_a())

1.介绍 1.介绍/
1.1.目标 1.介绍/1.1.目标/
1.2.快速开始 1.介绍/1.2.快速开始/
1.3.什么是计算机科学 1.介绍/1.3.什么是计算机科学/
1.4.什么是编程 1.介绍/1.4.什么是编程/
1.5.为什么要学习数据结构和抽象数据类型 1.介绍/1.5.为什么要学习数据结构和抽象数据类型/
1.6.为什么要学习算法 1.介绍/1.6.为什么要学习算法/
1.7.回顾Python基础 1.介绍/1.7.回顾Python基础/
2.算法分析 2.算法分析/
2.1.目标 2.算法分析/2.1.目标/
2.2.什么是算法分析 2.算法分析/2.2.什么是算法分析/
2.3.大O符号 2.算法分析/2.3.大O符号/
2.4.一个乱序字符串检查的例子 2.算法分析/2.4.一个乱序字符串检查的例子/
2.5.Python数据结构的性能 2.算法分析/2.5.Python数据结构的性能/
2.6.列表 2.算法分析/2.6.列表/
2.7.字典 2.算法分析/2.7.字典/
2.8.总结 2.算法分析/2.8.总结/
3.基本数据结构 3.基本数据结构/
3.1.目标 3.基本数据结构/3.1.目标/
3.2.什么是线性数据结构 3.基本数据结构/3.2.什么是线性数据结构/
3.3.什么是栈 3.基本数据结构/3.3.什么是栈/
3.4.栈的抽象数据类型 3.基本数据结构/3.4.栈的抽象数据类型/
3.5.Python实现栈 3.基本数据结构/3.5.Python实现栈/
3.6.简单括号匹配 3.基本数据结构/3.6.简单括号匹配/
3.7.符号匹配 3.基本数据结构/3.7.符号匹配/
3.8.十进制转换成二进制 3.基本数据结构/3.8.十进制转换成二进制/
3.9.中缀前缀和后缀表达式 3.基本数据结构/3.9.中缀前缀和后缀表达式/
3.10.什么是队列 3.基本数据结构/3.10.什么是队列/
3.11.队列抽象数据类型 3.基本数据结构/3.11.队列抽象数据类型/
3.12.Python实现队列 3.基本数据结构/3.12.Python实现队列/
bs_html:  <!DOCTYPE HTML>
<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="

bs_html:  <!DOCTYPE HTML>
<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>3.4.栈的抽象数据类型 · python-data-structure-cn</title>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="" name="description"/>
<meta content="GitBook 3.2.2" name="generator"/>
<meta content="facert" name="author"/>
<link href="../../gitbook/style.css" rel="stylesheet"/>
<link href="../../gitbook/gitbook-plugin-comment/plugin.css" rel="stylesheet"/>
<link href="../../gitbook/gitbook-plugin-highlight/website.css" rel="stylesheet"/>
<link href="../../gitbook/gitbook-plugin-search/search.css" rel="stylesheet"/>
<link href="../../gitbook/gitbook-plugin-fontsettings/website.css" rel="stylesheet"/>
<meta content="true" name="HandheldFriendly"/>
<meta content="width=device-width, initial-scale=1, user-scalable=no" name="viewport"/>
<meta content="yes" name="apple-mobile-web-app-capable"/>
<meta content="black" name="apple-mobile-web

bs_html:  <!DOCTYPE HTML>
<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>1.3.什么是计算机科学 · python-data-structure-cn</title>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="" name="description"/>
<meta content="GitBook 3.2.2" name="generator"/>
<meta content="facert" name="author"/>
<link href="../../gitbook/style.css" rel="stylesheet"/>
<link href="../../gitbook/gitbook-plugin-comment/plugin.css" rel="stylesheet"/>
<link href="../../gitbook/gitbook-plugin-highlight/website.css" rel="stylesheet"/>
<link href="../../gitbook/gitbook-plugin-search/search.css" rel="stylesheet"/>
<link href="../../gitbook/gitbook-plugin-fontsettings/website.css" rel="stylesheet"/>
<meta content="true" name="HandheldFriendly"/>
<meta content="width=device-width, initial-scale=1, user-scalable=no" name="viewport"/>
<meta content="yes" name="apple-mobile-web-app-capable"/>
<meta content="black" name="apple-mobile-web


bs_html:  <!DOCTYPE HTML>
<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>3.6.简单括号匹配 · python-data-structure-cn</title>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="" name="description"/>
<meta content="GitBook 3.2.2" name="generator"/>
<meta content="facert" name="author"/>
<link href="../../gitbook/style.css" rel="stylesheet"/>
<link href="../../gitbook/gitbook-plugin-comment/plugin.css" rel="stylesheet"/>
<link href="../../gitbook/gitbook-plugin-highlight/website.css" rel="stylesheet"/>
<link href="../../gitbook/gitbook-plugin-search/search.css" rel="stylesheet"/>
<link href="../../gitbook/gitbook-plugin-fontsettings/website.css" rel="stylesheet"/>
<meta content="true" name="HandheldFriendly"/>
<meta content="width=device-width, initial-scale=1, user-scalable=no" name="viewport"/>
<meta content="yes" name="apple-mobile-web-app-capable"/>
<meta content="black" name="apple-mobile-web-

bs_html:  <!DOCTYPE HTML>
<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>3.8.十进制转换成二进制 · python-data-structure-cn</title>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="" name="description"/>
<meta content="GitBook 3.2.2" name="generator"/>
<meta content="facert" name="author"/>
<link href="../../gitbook/style.css" rel="stylesheet"/>
<link href="../../gitbook/gitbook-plugin-comment/plugin.css" rel="stylesheet"/>
<link href="../../gitbook/gitbook-plugin-highlight/website.css" rel="stylesheet"/>
<link href="../../gitbook/gitbook-plugin-search/search.css" rel="stylesheet"/>
<link href="../../gitbook/gitbook-plugin-fontsettings/website.css" rel="stylesheet"/>
<meta content="true" name="HandheldFriendly"/>
<meta content="width=device-width, initial-scale=1, user-scalable=no" name="viewport"/>
<meta content="yes" name="apple-mobile-web-app-capable"/>
<meta content="black" name="apple-mobile-we

bs_html:  <!DOCTYPE HTML>
<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>3.9.中缀前缀和后缀表达式 · python-data-structure-cn</title>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="" name="description"/>
<meta content="GitBook 3.2.2" name="generator"/>
<meta content="facert" name="author"/>
<link href="../../gitbook/style.css" rel="stylesheet"/>
<link href="../../gitbook/gitbook-plugin-comment/plugin.css" rel="stylesheet"/>
<link href="../../gitbook/gitbook-plugin-highlight/website.css" rel="stylesheet"/>
<link href="../../gitbook/gitbook-plugin-search/search.css" rel="stylesheet"/>
<link href="../../gitbook/gitbook-plugin-fontsettings/website.css" rel="stylesheet"/>
<meta content="true" name="HandheldFriendly"/>
<meta content="width=device-width, initial-scale=1, user-scalable=no" name="viewport"/>
<meta content="yes" name="apple-mobile-web-app-capable"/>
<meta content="black" name="apple-mobile-w

bs_html:  <!DOCTYPE HTML>
<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>2.2.什么是算法分析 · python-data-structure-cn</title>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="" name="description"/>
<meta content="GitBook 3.2.2" name="generator"/>
<meta content="facert" name="author"/>
<link href="../../gitbook/style.css" rel="stylesheet"/>
<link href="../../gitbook/gitbook-plugin-comment/plugin.css" rel="stylesheet"/>
<link href="../../gitbook/gitbook-plugin-highlight/website.css" rel="stylesheet"/>
<link href="../../gitbook/gitbook-plugin-search/search.css" rel="stylesheet"/>
<link href="../../gitbook/gitbook-plugin-fontsettings/website.css" rel="stylesheet"/>
<meta content="true" name="HandheldFriendly"/>
<meta content="width=device-width, initial-scale=1, user-scalable=no" name="viewport"/>
<meta content="yes" name="apple-mobile-web-app-capable"/>
<meta content="black" name="apple-mobile-web-

bs_html:  <!DOCTYPE HTML>
<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>3.1.目标 · python-data-structure-cn</title>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="" name="description"/>
<meta content="GitBook 3.2.2" name="generator"/>
<meta content="facert" name="author"/>
<link href="../../gitbook/style.css" rel="stylesheet"/>
<link href="../../gitbook/gitbook-plugin-comment/plugin.css" rel="stylesheet"/>
<link href="../../gitbook/gitbook-plugin-highlight/website.css" rel="stylesheet"/>
<link href="../../gitbook/gitbook-plugin-search/search.css" rel="stylesheet"/>
<link href="../../gitbook/gitbook-plugin-fontsettings/website.css" rel="stylesheet"/>
<meta content="true" name="HandheldFriendly"/>
<meta content="width=device-width, initial-scale=1, user-scalable=no" name="viewport"/>
<meta content="yes" name="apple-mobile-web-app-capable"/>
<meta content="black" name="apple-mobile-web-app-s

In [51]:
e.queue

<Queue at 0x242ebee85c0 maxsize=0 _queue=[('1.介绍', '1.介绍/'), ('1.1.目标', '1.介绍/1.1.目标/'), ('1.2.快速开始', '1.介绍/1.2.快速开始/'), ('1.3.什么是计算机科学', '1.介绍/1.3.什么是计算机科学/'), ('1.4.什么是编程', '1.介绍/1.4.什么是编程/'), ('1.5.为什么要学习数据结构和抽象数据类型', '1.介绍/1.5.为什么要学习数据结构和抽象数据类型/'), ('1.6.为什么要学习算法', '1.介绍/1.6.为什么要学习算法/'), ('1.7.回顾Python基础', '1.介绍/1.7.回顾Python基础/'), ('2.算法分析', '2.算法分析/'), ('2.1.目标', '2.算法分析/2.1.目标/'), ('2.2.什么是算法分析', '2.算法分析/2.2.什么是算法分析/'), ('2.3.大O符号', '2.算法分析/2.3.大O符号/'), ('2.4.一个乱序字符串检查的例子', '2.算法分析/2.4.一个乱序字符串检查的例子/'), ('2.5.Python数据结构的性能', '2.算法分析/2.5.Python数据结构的性能/'), ('2.6.列表', '2.算法分析/2.6.列表/'), ('2.7.字典', '2.算法分析/2.7.字典/'), ('2.8.总结', '2.算法分析/2.8.总结/'), ('3.基本数据结构', '3.基本数据结构/'), ('3.1.目标', '3.基本数据结构/3.1.目标/'), ('3.2.什么是线性数据结构', '3.基本数据结构/3.2.什么是线性数据结构/'), ('3.3.什么是栈', '3.基本数据结构/3.3.什么是栈/'), ('3.4.栈的抽象数据类型', '3.基本数据结构/3.4.栈的抽象数据类型/'), ('3.5.Python实现栈', '3.基本数据结构/3.5.Python实现栈/'), ('3.6.简单括号匹配', '3.基本数据结构/3.6.简单括号匹配/'), ('3.7.符号匹配', '3.基本数据结构/3.7.符号匹配/'), ('3.8.十进制转换成二进制', '3.基本数据结构/3.8.十进制转换

# 新的异步代码

In [1]:
import asyncio, aiohttp
from asyncio import Queue
from bs4 import BeautifulSoup
from lxml import etree
import requests
import aiofiles
import re
import async_timeout

class aioEbook:
    def __init__(self, loop):
        self.loop = loop
        self.queue = Queue()
        self.start_url = "https://facert.gitbooks.io/python-data-structure-cn/"

    async def find_all_a(self):
        async with aiohttp.ClientSession() as session:
            async with session.get(self.start_url) as resp:
                content = await resp.text()
                html = etree.HTML(content)
                all_a_text = [i.strip() for i in html.xpath("//li[@class='chapter ']/a/text()")]
                all_a = html.xpath("//li[@class='chapter ']/a/@href")
                for a_text, a_href in zip(all_a_text, all_a):
                    self.queue.put_nowait((a_text, a_href))

        tasks = [asyncio.Task(self.download_html(self.queue, self.loop)) for _ in range(30)]
        await self.queue.join()
        for t in tasks:
            t.cancel()

    async def download_html(self, queue, loop):
        while queue.full():
            try:
                text, url = await queue.get()
                print(text, url)

                with async_timeout.timeout(30):
                    async with aiofiles.open('html/' + text + '.html', 'w', encoding='utf8') as f:

                        async with aiohttp.ClientSession(loop=loop) as session:
                            async with session.get(self.start_url + url) as resp:
                                resp_text = await resp.text()
                                bs_html = BeautifulSoup(resp_text, 'lxml')
                                print('bs_html: ', bs_html)
                                content = str(bs_html.find("section", 'normal markdown-section'))
                                filter_href = re.sub(r'href="(?!http)', 'href="' + url, content)
                                filter_src = re.sub(r'src="(?!http)', 'src="' + url, filter_href)
                                print('filter_src: ', filter_src)

                        await f.write(filter_src)

                print('qsize(): ', self.queue.qsize())
                queue.task_done()

            except asyncio.CancelledError:
                pass
            except Exception as e:
                print('error: ', e)


#loop = asyncio.get_event_loop()
#e = aioEbook(loop)
#loop.run_until_complete(e.find_all_a())

In [ ]:
htmls = os.listdir('html')
htmls = ['html/'+html for html in htmls]
options = {
    'page-size': 'Letter',
    'margin-top': '0.75in',
    'margin-right': '0.75in',
    'margin-bottom': '0.75in',
    'margin-left': '0.75in',
    'encoding': "UTF-8",
    'custom-header' : [
        ('Accept-Encoding', 'gzip')
    ]
    'cookie': [
        ('cookie-name1', 'cookie-value1'),
        ('cookie-name2', 'cookie-value2'),
    ],
    'no-outline': None
}
import pdfkit
pdfkit.from_file(htmls, 'python数据结构与算法中文.pdf', options={'encoding':'utf8'})
